In [ ]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from category_encoders import TargetEncoder
import os


In [ ]:
# Settings for cleaner output
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
#Load the raw dataset
df=pd.read_csv('../data/raw/Melbourne_housing_FULL.csv')
#Basic check
df.head()

In [ ]:
#Data types
print(df.info())

In [ ]:
#Convert the 'Date' feature to datetime object
df['Date']=pd.to_datetime(df['Date'], dayfirst=True)

#Verification
print(df.info())

In [ ]:
#Duplicate check 
print("Number of duplicate rows:", df.duplicated().sum())
#Show the duplicated rows
df[df.duplicated()]

In [ ]:
#Remove duplicated rows
df=df.drop_duplicates(keep='first').reset_index(drop=True)

#Verification
print("Number of duplicate rows after cleaning :",df.duplicated().sum())

In [ ]:
#Sanity check 
display(df.describe().T)

In [ ]:
#Check how many rows (buildings) that have Building Area less than 5 or greater than 10000
print("Number of Buildings that have Building Area less than 5:", len(df[df['BuildingArea'] < 5]))
print("Number of Buildings that have Building Area more than 10000:", len(df[df['BuildingArea'] > 10000]))

#Replace those impossible values with NaN 
df.loc[df['BuildingArea'] < 5, 'BuildingArea'] = np.nan
df.loc[df['BuildingArea'] > 10000, 'BuildingArea'] = np.nan

#Verification
print("Number of Buildings that have Building Area less than 5 after cleaning:", len(df[df['BuildingArea'] < 5]))
print("Number of Buildings that have Building Area more than 10000 after cleaning:", len(df[df['BuildingArea'] > 10000]))

#Number of missing values in the feature
print("Number of missing values :",df['BuildingArea'].isnull().sum())

In [ ]:
#Find impossible YearBuilt values

#Get the dynamic current year
current_year=datetime.datetime.now().year

#Find count of properties that build before 1800 and after ccurrent year
print("Number of properties with impossible YearBuilt values:", ((df['YearBuilt'] < 1800) | (df['YearBuilt'] >current_year)).sum())

#Fix the impossible YearBuilt values by replacing them with NaN
df['YearBuilt'] = df['YearBuilt'].where(df['YearBuilt'].between(1800, current_year), np.nan)

#Verification
print("Number of properties with impossible YearBuilt values after cleaning:", ((df['YearBuilt'] < 1800) | (df['YearBuilt'] > current_year)).sum())

#Number of missing value in the feature
print("Number of missing values :", df['YearBuilt'].isnull().sum())

In [ ]:
#Check impossible properties with 0 bathrooms
print("Count of properties with 0 bathrooms:", (df['Bathroom'] == 0).sum())

#Logic of replacing 0 bathrooms with NaN

#If the bathroom is 0 and the property type is house and the year built is after 1900, then it is likely an error
df.loc[(df['Type'] == 'h') & (df['YearBuilt'] > 1900) & (df['Bathroom'] == 0), 'Bathroom'] = np.nan

#If the bathroom is 0 and the property type is unit or townhouse, then it is likely an error
df.loc[(df['Type'].isin(['u', 't'])) & (df['Bathroom'] == 0), 'Bathroom'] = np.nan

#Verification
print("Count of properties with 0 bathrooms after cleaning:", (df['Bathroom'] == 0).sum())

#Number of missing values in the feature
print("Number of missing values :", df['Bathroom'].isnull().sum())


In [ ]:
#Inconsistency Scan
text_features = df.select_dtypes(include=['object']).columns
for col in text_features:
    unique_count=df[col].nunique()
    print(f"Feature [{col}] has {unique_count} unique values.")
    print(sorted(df[col].unique().astype(str)))
    print("\n")

In [ ]:
#Fix the inconsistencies

# Convert the 'Suburb' feature to title case
df['Suburb'] = df['Suburb'].str.title()

#Convert 'SellerG' to remove branch info and fix casing
df['SellerG'] = df['SellerG'].str.split('/').str[0].str.strip().str.title()

#Verfication
print("--- Suburb ---")
print(sorted(df[df['Suburb'].str.contains('Croydon', case=False)]['Suburb'].unique()))
print(sorted(df[df['Suburb'].str.contains('Viewbank', case=False)]['Suburb'].unique()))

print("\n--- SellerG ---")
brands_to_check = ['Buxton', 'Hockingstuart', 'Vicprop']
for brand in brands_to_check:
    print(f"{brand} variations: {sorted([s for s in df['SellerG'].unique() if brand.lower() in s.lower()])}")



In [ ]:
#Calculate number of missing values per feature
print("Number of missing values per feature: \n", df.isnull().sum().sort_values(ascending=False))

In [ ]:
#Drop the missing values of target feature 
df=df.dropna(subset=['Price'])

#Drop the missing values of the Postcode feature
df=df.dropna(subset=['Postcode'])

#Verification 
print("Number of missing values per feature after dropping target and Postcode missing values:\n", df.isnull().sum().sort_values(ascending=False))

In [ ]:
# --- 5. Data Splitting Strategy ---
# 1. Define Features (X) and Target (y)
X = df.drop('Price', axis=1)
y = df['Price']

# 2. Perform Random Split (User Decision: Random Split for Generalization)
# shuffle=True ensures we mix 2016, 2017, and 2018 data together
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)



# 3. Verify
print("--- Random Split Successful ---")
print(f"Training Set: {X_train.shape[0]} rows")
print(f"Test Set:     {X_test.shape[0]} rows")

# 4. SAFETY CHECK
assert len(X_train) + len(X_test) == len(df), "Error: Row mismatch!"
print("Safety Check Passed.")

In [ ]:
#Feature Engineering

#Extract the Sold Year from the 'Date' feature
X_train['SoldYear'] = pd.to_datetime(X_train['Date']).dt.year
X_test['SoldYear'] = pd.to_datetime(X_test['Date']).dt.year

In [ ]:
#Drop Features

features_to_drop = ['Date', 'Address', 'Bedroom2', 'CouncilArea', 'Method', 'Postcode']

X_train = X_train.drop(columns=features_to_drop, errors='ignore')
X_test = X_test.drop(columns=features_to_drop, errors='ignore')

In [ ]:
#Rare Category Grouping

#Group all specific rural regions into one "Regional Victoria" category.
regions_to_group = ['Eastern Victoria', 'Northern Victoria', 'Western Victoria']

#Apply to training and test sets
X_train['Regionname'] = X_train['Regionname'].replace(regions_to_group, 'Regional Victoria')
X_test['Regionname'] = X_test['Regionname'].replace(regions_to_group, 'Regional Victoria')

#Verification
print(X_train['Regionname'].value_counts())


In [ ]:
X_train.isna().sum().sort_values(ascending=False)

In [ ]:
#Handeling Missing values (Imputation)

#Simple Imputations
# A. Regionname (Categorical) -> Use MODE (Most Frequent)
# We use [0] to get the first value if there's a tie
mode_region = X_train['Regionname'].mode()[0]
X_train['Regionname'] = X_train['Regionname'].fillna(mode_region)
X_test['Regionname'] = X_test['Regionname'].fillna(mode_region)

# B. Propertycount (Numerical) -> Use MEDIAN
median_prop = X_train['Propertycount'].median()
X_train['Propertycount'] = X_train['Propertycount'].fillna(median_prop)
X_test['Propertycount'] = X_test['Propertycount'].fillna(median_prop)

In [ ]:
# Advanced Imputation for 'Lattitude' and 'Longtitude' using Suburb Centroids

# --- STEP 1: Compute suburb centroids from training data ---
suburb_coords = X_train.groupby('Suburb')[['Lattitude', 'Longtitude']].mean()

# Count how many houses each suburb has
suburb_counts = X_train['Suburb'].value_counts()

# Minimum number of houses to consider a suburb "reliable"
min_houses = 3
reliable_suburbs = suburb_counts[suburb_counts >= min_houses].index

# Robust centroids for reliable suburbs
robust_centroids = suburb_coords.loc[reliable_suburbs]

# Global median fallback
global_lat = X_train['Lattitude'].median()
global_long = X_train['Longtitude'].median()

# --- STEP 2: Map centroids for TRAINING SET ---

# Map reliable centroids, leave others as NaN
train_lat = X_train['Suburb'].map(robust_centroids['Lattitude'])
train_long = X_train['Suburb'].map(robust_centroids['Longtitude'])

# Fill only missing values with mapped centroid
X_train['Lattitude'] = X_train['Lattitude'].fillna(train_lat)
X_train['Longtitude'] = X_train['Longtitude'].fillna(train_long)

# Fill any remaining NaNs with global median
X_train['Lattitude'] = X_train['Lattitude'].fillna(global_lat)
X_train['Longtitude'] = X_train['Longtitude'].fillna(global_long)

# --- STEP 3: Map centroids for TEST SET ---

# Map centroids from training data
test_lat = X_test['Suburb'].map(robust_centroids['Lattitude'])
test_long = X_test['Suburb'].map(robust_centroids['Longtitude'])

# Fill missing values with mapped centroid
X_test['Lattitude'] = X_test['Lattitude'].fillna(test_lat)
X_test['Longtitude'] = X_test['Longtitude'].fillna(test_long)

# Fill any remaining NaNs with global median
X_test['Lattitude'] = X_test['Lattitude'].fillna(global_lat)
X_test['Longtitude'] = X_test['Longtitude'].fillna(global_long)

print("✅ Vectorized Advanced Suburb Centroid Imputation Complete for Train & Test")


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
import numpy as np
import pandas as pd

# --- STEP 1: Encode Categoricals (Strings -> Integers) ---
# We keep the encoder to reverse this later if needed
cat_cols = X_train.select_dtypes(include=['object']).columns
ord_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train[cat_cols] = ord_encoder.fit_transform(X_train[cat_cols])
X_test[cat_cols] = ord_encoder.transform(X_test[cat_cols])

# --- STEP 2: SCALING (Crucial for KNN) ---
# Squash everything to 0-1 so 'Landsize' doesn't overpower 'Car'
scaler = MinMaxScaler()

# Scale and keep as DataFrame (to track columns)
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# --- STEP 3: KNN Imputation ---
# KNN finds neighbors using the scaled data
knn_imputer = KNNImputer(n_neighbors=5)

X_train_imputed = pd.DataFrame(knn_imputer.fit_transform(X_train_scaled), columns=X_train.columns, index=X_train.index)
X_test_imputed = pd.DataFrame(knn_imputer.transform(X_test_scaled), columns=X_test.columns, index=X_test.index)

# --- STEP 4: INVERSE TRANSFORM & CLEANUP ---
# 1. Bring everything back to original scale (Year=2017, Region=5.0)
X_train_final = pd.DataFrame(scaler.inverse_transform(X_train_imputed), columns=X_train.columns, index=X_train.index)
X_test_final = pd.DataFrame(scaler.inverse_transform(X_test_imputed), columns=X_test.columns, index=X_test.index)

# 2. ROUND Categoricals (The Critical Fix)
# Inverse transform might give 4.9999 instead of 5. We must round to nearest integer.
# We have to combine categorical columns + Numerical columns for this step (Didnt include Distance,BuildingArea etc. as they can be float)

cols_to_round = list(cat_cols) + ['Bathroom', 'Car', 'YearBuilt', 'Rooms', 'Propertycount', 'SoldYear']

for col in cols_to_round:
    if col in X_train_final.columns:
        # Round to nearest whole number and cast to Integer
        X_train_final[col] = X_train_final[col].round(0).astype(int)
        X_test_final[col] = X_test_final[col].round(0).astype(int)


# 3. Update main variables
X_train = X_train_final
X_test = X_test_final

print("✅ Imputation Complete.")
print(f"Missing Values: \n{X_train.isnull().sum()}")

In [ ]:
X_train.head()

In [ ]:
# --- SAFETY CHECK: Verify no negatives before Log Transform ---
skewed_features = ['Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Propertycount']

# Check for negatives in Train and Test
negatives_train = (X_train[skewed_features] < 0).sum().sum()
negatives_test = (X_test[skewed_features] < 0).sum().sum()

if negatives_train == 0 and negatives_test == 0:
    print("✅ Safe to Proceed: All features are non-negative.")
else:
    print(f"⚠️ STOP! Found {negatives_train} negatives in Train and {negatives_test} in Test.")
    print("   Do NOT apply Log Transform to negative values. Check if you scaled too early.")

In [ ]:
#Transforming Skewed Features with LogTransform

#Target Transformation
# We transform y_train so the model learns a Normal Distribution
# We transform y_test ONLY for checking error metrics later (Optional but good practice)
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

#Feature Transformation
skewed_features = ['Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Propertycount']

#Apply np.log1p to these features in both Train and Test
for col in skewed_features:
    if col in X_train.columns:
        X_train[col] = np.log1p(X_train[col])
        X_test[col] = np.log1p(X_test[col])
        print("Log transformed :", col)

In [ ]:
#Scaling Features

# --- DEFINE GROUPS ---
robust_features = ['Rooms', 'Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Propertycount']
standard_features = ['YearBuilt', 'Lattitude', 'Longtitude']

# --- STEP: CHECK FOR ZERO VARIANCE / ZERO IQR ---
# We need to know if RobustScaler or StandardScaler will fail silently.

# 1. Check Standard Deviation (for StandardScaler features)
std_check = X_train[standard_features].std()
constant_std = std_check[std_check == 0].index.tolist()

if constant_std:
    print(f"⚠️ WARNING: The following columns have ZERO variance (Constant): {constant_std}")
    print("   Action: Drop these columns before scaling, or the model will ignore them.")
else:
    print("✅ StandardScaler Safety Check Passed (No constant columns).")

# 2. Check IQR (for RobustScaler features)
# Calculate Q1 (25%) and Q3 (75%)
Q1 = X_train[robust_features].quantile(0.25)
Q3 = X_train[robust_features].quantile(0.75)
IQR = Q3 - Q1

# Find columns where IQR is 0
zero_iqr = IQR[IQR == 0].index.tolist()

if zero_iqr:
    print(f"⚠️ WARNING: The following columns have an IQR of 0: {zero_iqr}")
else:
    print("✅ RobustScaler Safety Check Passed (No zero-IQR columns).")

In [ ]:
#ROBUST SCALER
scaler_rob = RobustScaler()

# We use .loc to ensure we update the specific columns in place
# Check if columns exist to prevent errors
existing_robust = [col for col in robust_features if col in X_train.columns]

if existing_robust:
    X_train[existing_robust] = scaler_rob.fit_transform(X_train[existing_robust])
    X_test[existing_robust] = scaler_rob.transform(X_test[existing_robust])
    print(f"✅ Applied RobustScaler to: {existing_robust}")

#STANDARD SCALER
scaler_std = StandardScaler()

existing_std = [col for col in standard_features if col in X_train.columns]

if existing_std:
    X_train[existing_std] = scaler_std.fit_transform(X_train[existing_std])
    X_test[existing_std] = scaler_std.transform(X_test[existing_std])
    print(f"✅ Applied StandardScaler to: {existing_std}")

In [ ]:

# --- PART 1: ONE-HOT ENCODING (Low Cardinality) ---
# Features: 'Type', 'Regionname'
# Logic: Best for categories with few options (<10).
# Industrial Standard: Use OneHotEncoder(handle_unknown='ignore') to safely handle new categories in Test.

ohe_cols = ['Type', 'Regionname']

# 1. Setup Encoder
# sparse_output=False creates a regular pandas DataFrame, not a compressed matrix.
# dtype=int makes the output 0/1 instead of 0.0/1.0.
encoder_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=int).set_output(transform='pandas')

# 2. Fit & Transform
# We fit only on Train to learn the categories.
X_train_ohe = encoder_ohe.fit_transform(X_train[ohe_cols])
X_test_ohe = encoder_ohe.transform(X_test[ohe_cols])

# 3. Merge & Clean
# Attach new columns and drop the old text columns
X_train = pd.concat([X_train, X_train_ohe], axis=1).drop(columns=ohe_cols)
X_test = pd.concat([X_test, X_test_ohe], axis=1).drop(columns=ohe_cols)

print(f"✅ One-Hot Encoding Complete. Added {X_train_ohe.shape[1]} binary columns.")


# --- PART 2: TARGET ENCODING (High Cardinality) ---
# Features: 'Suburb', 'SellerG'
# Logic: Best for categories with hundreds of options.
# Industrial Standard: Use 'category_encoders' with smoothing to prevent overfitting on rare suburbs.

target_cols = ['Suburb', 'SellerG']

# 1. Setup Encoder
# smoothing=10.0: Higher values mean we trust the "Global Average" more for rare categories.
# This is the "Safety Net" that manual coding often misses.
encoder_target = TargetEncoder(cols=target_cols, smoothing=10.0)

# 2. Fit & Transform
# CRITICAL: We fit on X_train and y_train_log.
# The model learns the relationship between Suburb -> Log Price.
X_train[target_cols] = encoder_target.fit_transform(X_train[target_cols], y_train_log)

# 3. Transform Test
# We use the learned patterns to transform Test (no peeking at y_test!)
X_test[target_cols] = encoder_target.transform(X_test[target_cols])

print(f"✅ Target Encoding (Industrial) Complete on: {target_cols}")


# --- PART 3: FINAL SAFETY CHECK (Crucial) ---
# Industrial Standard: Ensure Test set has exact same columns in exact same order.
# This prevents "Silent Failures" where columns get swapped.

# 1. Reindex Test to match Train
# Drops any extra columns in Test, adds missing ones (filling with 0), and fixes order.
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 2. Strict Assertion
# If this line fails, the pipeline stops immediately.
assert list(X_train.columns) == list(X_test.columns), "❌ CRITICAL ERROR: Column mismatch between Train and Test!"

print("\n🚀 Encoding & Alignment Successful.")
print(f"Final Data Shape: {X_train.shape}")

In [ ]:

# --- FINAL STEP: SAVE PROCESSED DATA ---
# We save in 'parquet' format because it preserves your integer columns (Car, YearBuilt).
# CSV is bad because it might turn them back into text or floats.

save_path = '../data/processed/'
os.makedirs(save_path, exist_ok=True)

print("💾 Saving Processed Data...")

# Save Features
X_train.to_parquet(f'{save_path}X_train_processed.parquet')
X_test.to_parquet(f'{save_path}X_test_processed.parquet')

# Save Targets
# We convert to DataFrame because Series cannot be saved as Parquet directly
pd.DataFrame(y_train_log, columns=['Price']).to_parquet(f'{save_path}y_train_log.parquet')
pd.DataFrame(y_test_log, columns=['Price']).to_parquet(f'{save_path}y_test_log.parquet')

# Optional: Save the "Real Dollar" y_test for final error checking later
pd.DataFrame(y_test, columns=['Price']).to_parquet(f'{save_path}y_test_real.parquet')

print(f"✅ success! Processed data saved to: {save_path}")